## Elastic Search

In [ ]:
%%capture

!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [ ]:
import os
import elasticsearch
from elasticsearch import Elasticsearch
import sys
import json
from ast import literal_eval
from tqdm import tqdm
import datetime
from elasticsearch import helpers

In [ ]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Creating daemon instance of elasticsearch
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(60)

In [ ]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon      1890    1077 56 12:00 ?        00:00:33 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g
daemon      1993    1890  0 12:00 ?        00:00:00 /content/elasticsearch-7.0.0/modules/x-pack-ml/p
root        2255    2253  0 12:01 ?        00:00:00 grep elasticsearch


In [ ]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

{
  "name" : "e96b2f188f7a",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "YCdtpFr6QwWb_o8wGuE_VA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

True

In [ ]:
Settings = {
"mappings":{
    "properties":{
        "article":{
            "type":"text",
            'similarity': 'BM25'
        }
    }
}}

In [ ]:
def json_formatter(dataset, index_name, index_type='_doc'):
    """
    This function is used to create JSON formatted dictionaries for Elasticsearch.

    Args:
      dataset: The dataset you want to apply this function.
      index_name: Name of the index in Elasticsearch
      index_type: Type of the index in Elasticsearch.
      Note: It is suggested to keep index_type as '_doc' since it is deprecated from version 6.
      Note: This function formats all columns of your dataset, if you want to apply this to special columns only,
      you can delete the second for loop and add your custom fields.
    """
    try:
        List = []

        for idx, row in enumerate(dataset):
            dic = {}
            dic['_index'] = index_name
            dic['sentence'] = row
            List.append(dic)
        return List

    except Exception as e:
        print("There is a problem: {}".format(e))

In [ ]:
MY_INDEX = es.indices.create(index="corpus_arc", ignore=[400,404], body=Settings)
MY_INDEX

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'corpus_arc'}

In [ ]:
MY_INDEX = es.indices.create(index="corpus_aristo", ignore=[400,404], body=Settings)
MY_INDEX

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'corpus_aristo'}

In [ ]:
index_exists = es.indices.exists(index="corpus_arc")

# Print the result
if index_exists:
    print(f"The index corpus_arc exists.")
else:
    print(f"The index corpus_arc does not exist.")

In [ ]:
arc_json_formatted = json_formatter(dataset=corpus_arc, index_name='corpus_arc', index_type='_doc')

aristo_json_formatted = json_formatter(dataset=corpus_aristo, index_name='corpus_aristo', index_type='_doc')


In [ ]:
res_arc = helpers.bulk(es, arc_json_formatted)
res_aristo= helpers.bulk(es, aristo_json_formatted)

In [ ]:
output = pd.json_normalize((response['hits']['hits']))
output

,_index,_type,_id,_score,_source.sentence
0,corpus_arc,_doc,ftJou4oBxymN87z6RhKX,15.544049,dried parsley cleaned nutella container
1,corpus_arc,_doc,cMZnu4oBxymN87z6RrVw,10.175644,nutella creamy chocolate hazelnut combo terrific occasional treatbut hardly part balanced breakfast commercials say
2,corpus_arc,_doc,cMVnu4oBxymN87z6M6O4,7.353719,recipes include wheat bread instead white nutella chocolate spread instead peanut butter addition bananas honey ingredients chips potato chips corn chips etc inside sandwich
